<a href="https://colab.research.google.com/github/btoneil2021/basketball-sql/blob/main/CollegeExtractionTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To-Do:
- Extract school names from big list
- Take a team name from a player
- Use rapidfuzz to map the two

# Part 1 - Extract School Names from Big List!

In [1]:
# Extract Names from list (https://www.ncsasports.org/mens-basketball/colleges)

In [2]:
%%shell
pip install chromedriver_autoinstaller
pip install selenium
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.3 MB/s eta 0:00:00


In [3]:
import sys
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import threading
import unicodedata
import rapidfuzz

In [4]:
def get_soup(url):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=chrome_options)

  driver.get(url)

  driver.implicitly_wait(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  driver.quit()

  return soup

In [5]:
soup = get_soup('https://www.ncsasports.org/mens-basketball/colleges')

In [6]:
college_table = soup.find('section', {'class': 'wp-block-ncsa-college-list'})

In [7]:
college_rows = college_table.find_all('div', {'itemprop': 'itemListElement'})

In [18]:
colleges = []
college_and_division = {}
for div in college_rows:
  name_div = div.find('div', {'itemprop': 'name'})
  division_div = div.find_all('div')[-1].get_text(strip=True)
  name = name_div.text
  colleges.append(name)
  college_and_division[name] = division_div

In [19]:
print(colleges)

['Abilene Christian University', 'Academy of Art University', 'Adams State University', 'Adelphi University', 'Adrian College', 'Alabama A&M University', 'Alabama State University', 'Albany State University', 'Albany Technical College', 'Albertus Magnus College', 'Albion College', 'Albright College', 'Alcorn State University', 'Alderson Broaddus University', 'Alfred University', 'Alice Lloyd College', 'Allan Hancock College', 'Allegany College of Maryland', 'Allegheny College', 'Allen Community College', 'Allen University', 'Alliance University', 'Alma College', 'Alpena Community College', 'Alvernia University', 'American International College', 'American River College', 'American University', 'Amherst College', 'Anderson University – Indiana', 'Anderson University – South Carolina', 'Andrew College', 'Angelina College', 'Angelo State University', 'Anna Maria College', 'Anne Arundel Community College', 'Anoka-Ramsey Community College', 'Appalachian State University', 'Aquinas College –

In [20]:
print(college_and_division)

{'Abilene Christian University': 'NCAA D1', 'Academy of Art University': 'NCAA D2', 'Adams State University': 'NCAA D2', 'Adelphi University': 'NCAA D2', 'Adrian College': 'NCAA D3', 'Alabama A&M University': 'NCAA D1', 'Alabama State University': 'NCAA D1', 'Albany State University': 'NCAA D2', 'Albany Technical College': 'JC', 'Albertus Magnus College': 'NCAA D3', 'Albion College': 'NCAA D3', 'Albright College': 'NCAA D3', 'Alcorn State University': 'NCAA D1', 'Alderson Broaddus University': 'NCAA D2', 'Alfred University': 'NCAA D3', 'Alice Lloyd College': 'NAIA', 'Allan Hancock College': 'JC', 'Allegany College of Maryland': 'JC', 'Allegheny College': 'NCAA D3', 'Allen Community College': 'JC', 'Allen University': 'NCAA D2', 'Alliance University': 'NCAA D2', 'Alma College': 'NCAA D3', 'Alpena Community College': 'JC', 'Alvernia University': 'NCAA D3', 'American International College': 'NCAA D2', 'American River College': 'JC', 'American University': 'NCAA D1', 'Amherst College': 'NC

# Part 3 - Use rapidfuzz to map the two together

In [10]:
"""
IDEAS FOR LATER! (SINCE IT DOESN'T COMPLETELY WORK RIGHT NOW)

- Grab the Division too, figure out what division each school in top 10 is in
  - Then take the most accurate that is also best at bball (D1, then D2, etc.)
"""

"\nIDEAS FOR LATER! (SINCE IT DOESN'T COMPLETELY WORK RIGHT NOW)\n\n- Grab the Division too, figure out what division each school in top 10 is in\n  - Then take the most accurate that is also best at bball (D1, then D2, etc.)\n"

In [34]:
def match_school(input_name, school_list, college_and_division_together, limit=10):
  top_x = rapidfuzz.process.extract(input_name, school_list, limit=limit)

  rankings = {
      5: 'NCAA D1',
      4: 'NCAA D2',
      3: 'NCAA D3',
      2: 'NAIA',
      1: 'JC'
  }

  best_college_rank = 0
  best_college = None
  for x in top_x:
    if college_and_division_together[x[0]] == 'NCAA D1' and best_college_rank < 5:
      best_college = x[0]
      best_college_rank = 5
    elif college_and_division_together[x[0]] == 'NCAA D2' and best_college_rank < 4:
      best_college = x[0]
      best_college_rank = 4
    elif college_and_division_together[x[0]] == 'NCAA D3' and best_college_rank < 3:
      best_college = x[0]
      best_college_rank = 3
    elif college_and_division_together[x[0]] == 'NAIA' and best_college_rank < 2:
      best_college = x[0]
      best_college_rank = 2
    elif college_and_division_together[x[0]] == 'JC' and best_college_rank < 1:
      best_college = x[0]
      best_college_rank = 1

  if best_college == 0:
    return None
  return best_college

In [35]:
print(match_school('Georgia', colleges, college_and_division))

Georgia Southern University


In [ ]:
# I have done a lot and dk where to go. Pin in this.